In [3]:
import cv2
import socket
import numpy as np
from ultralytics import YOLO
import threading

model = YOLO("yolov8n.pt")
#IP = "192.168.228.160"
#IP = "172.20.10.2"
IP = "10.42.0.1"
PORT = 8554

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((IP, PORT))

data = b""
latest_frame = None

def frame_reader():
    global data, latest_frame
    start_marker = b'\xff\xd8'
    end_marker = b'\xff\xd9'

    while True:
        chunk = sock.recv(4096)
        if not chunk:
            continue
        data += chunk

        start = data.find(start_marker)
        end = data.find(end_marker)

        if start != -1 and end != -1 and end > start:
            jpg = data[start:end+2]
            data = data[end+2:]
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            latest_frame = frame  # always overwrite with newest frame

reader_thread = threading.Thread(target=frame_reader, daemon=True)
reader_thread.start()

while True:
    if latest_frame is None:
        continue

    frame = cv2.rotate(latest_frame, cv2.ROTATE_180)

    # Run YOLO on most recent frame only
    results = model(frame, imgsz=480)
    annotated = results[0].plot()

    cv2.imshow("YOLO Stream", annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

sock.close()
cv2.destroyAllWindows()


0: 384x480 1 person, 70.1ms
Speed: 1.3ms preprocess, 70.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 40.8ms
Speed: 1.1ms preprocess, 40.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 36.4ms
Speed: 1.1ms preprocess, 36.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 39.0ms
Speed: 0.9ms preprocess, 39.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 36.2ms
Speed: 0.9ms preprocess, 36.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 38.2ms
Speed: 1.4ms preprocess, 38.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 36.7ms
Speed: 0.9ms preprocess, 36.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 1 person, 42.4ms
Speed: 0.9ms preprocess, 42.4ms inference, 0.5ms postprocess per image at shape (1, 3, 38

Exception in thread Thread-6 (frame_reader):
Traceback (most recent call last):
  File "/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure


    _threading_Thread_run(self)
  File "/Users/yahyarahhawi/miniforge3/envs/ml/lib/python3.12/threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/p3/zqj4hsr94qs443gkb32p5h_00000gn/T/ipykernel_41335/1050316098.py", line 25, in frame_reader
OSError: [Errno 9] Bad file descriptor
